In [3]:
word2index = load_pickle_file("../../../new_disk/word2index")
embeding_matrix = load_pickle_file("../../../new_disk/embeding_matrix")
index_sequences_titles = load_pickle_file("./index_sequences_titles.pkl")
index_sequences_text = load_pickle_file("../../../new_disk/index_sequences.pkl")
labels = load_pickle_file("../labels")
SEQUENCE_LENGTH = 434
SEQUENCE_LENGTH_TITLE = 11
VOCAB_SIZE = len(embeding_matrix)
EMB_DIMENSION = 300
VALIDATION_SPLIT = 0.1
PAD_SEQ_VALUE = 3000440

In [4]:
import numpy as np
same_lenght_seq = [seq[:SEQUENCE_LENGTH].reshape(1,SEQUENCE_LENGTH) if len(seq) >= SEQUENCE_LENGTH 
                   else np.concatenate((seq,np.array([PAD_SEQ_VALUE] * (SEQUENCE_LENGTH - len(seq)))),axis = 0).reshape(1,SEQUENCE_LENGTH) 
                   for seq in index_sequences_text]
same_lenght_seq = np.concatenate(same_lenght_seq,axis=0)

same_lenght_titles = [seq[:SEQUENCE_LENGTH_TITLE].reshape(1,SEQUENCE_LENGTH_TITLE) if len(seq) >= SEQUENCE_LENGTH_TITLE 
                   else np.concatenate((seq,np.array([PAD_SEQ_VALUE] * (SEQUENCE_LENGTH_TITLE - len(seq)))),axis = 0).reshape(1,SEQUENCE_LENGTH_TITLE) 
                   for seq in index_sequences_titles]
same_lenght_titles = np.concatenate(same_lenght_titles,axis=0)
same_lenght_titles = np.array([[int(x) for x in l] for l in same_lenght_titles])
labels = np.vstack(np.array([1,0]).reshape(1,2) if x == 0 else np.array([0,1]).reshape(1,2) for x in labels)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if sys.path[0] == '':


Dataset split

In [5]:
from sklearn.model_selection import train_test_split

x_train_indexes, x_test_indexes, y_train, y_test = train_test_split(list(range(len(same_lenght_seq))), 
                                                                                    labels, test_size=0.2,
                                                                                    random_state=42,
                                                                                    shuffle = True)
x_train_text = same_lenght_seq[x_train_indexes]
x_train_title = same_lenght_titles[x_train_indexes]

x_test_text = same_lenght_seq[x_test_indexes]
x_test_title = same_lenght_titles[x_test_indexes]

In [6]:
#create validation dataset
data_size = len(x_train_text)
perm = np.random.permutation(data_size)
idx_train = perm[:int(data_size*(1-VALIDATION_SPLIT))]
idx_val = perm[int(data_size*(1-VALIDATION_SPLIT)):]

data_train_text = x_train_text[idx_train]
data_train_title = x_train_title[idx_train]

labels_train = y_train[idx_train]

data_val_text = x_train_text[idx_val]
data_val_title = x_train_title[idx_val]

labels_val = y_train[idx_val]

In [13]:
data_train.shape

(7380, 434)

<h1>Convolutional neural networks</h1>

In [7]:
import tensorflow as tf
from tensorflow import keras as tk
from tensorflow.keras import layers as tfkl
from sklearn.metrics import mean_squared_error,mean_absolute_error

from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Conv1D, MaxPool1D, SpatialDropout1D, GlobalMaxPool1D, BatchNormalization
from tensorflow.keras.models import Model
# from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [7]:
model_checkpoint = ModelCheckpoint("./Checkpoints", save_best_only=True, save_weights_only=True)


In [36]:
data_train_text.shape

(11397, 618)

In [24]:
#Text model
LSTM_HIDDEN_DIM_SIZE = 8
NUM_FILTERS = 25
KERNEL_SIZE = 18
rate_drop_lstm = 0.15
rate_drop_lstm = 0.15
input_1 = Input(shape=(SEQUENCE_LENGTH,))
embedding_layer = Embedding(VOCAB_SIZE,
        EMB_DIMENSION,
        weights=[embeding_matrix],
        input_length=SEQUENCE_LENGTH,
        trainable=False)(input_1)
x1 = Conv1D(NUM_FILTERS, KERNEL_SIZE, activation = 'relu',kernel_regularizer =tf.keras.regularizers.l2(l=0.01))(embedding_layer)
x1 = Dropout(0.2)(x1)
x1 = MaxPool1D(pool_size = 15)(x1)
x1 = LSTM(LSTM_HIDDEN_DIM_SIZE, activation = 'tanh', dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)(embedding_layer)

#Titles model
NUM_FILTERS = 2
KERNEL_SIZE = 4
rate_drop_lstm = 0.15
rate_drop_lstm = 0.15

input_2 = Input(shape=(SEQUENCE_LENGTH_TITLE,))
embedding_layer = Embedding(VOCAB_SIZE,
        EMB_DIMENSION,
        weights=[embeding_matrix],
        input_length=SEQUENCE_LENGTH_TITLE,
        trainable=False)(input_2)


x2 = Conv1D(NUM_FILTERS, KERNEL_SIZE, activation = 'relu',kernel_regularizer =tf.keras.regularizers.l2(l=0.01))(embedding_layer)
# x2 = Dropout(0.2)(x2)
x2 = MaxPool1D(pool_size = 2)(x2)
x2 = GlobalMaxPool1D()(x2)

concat = tk.layers.Concatenate()([x1,x2])
output = Dense(2,activation='sigmoid')(concat)
full_model = tf.keras.Model(inputs=[input_1, input_2], outputs=[output])

In [25]:
full_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 11)]         0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 11, 300)      900132300   input_8[0][0]                    
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 434)]        0                                            
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 8, 2)         2402        embedding_7[0][0]                
____________________________________________________________________________________________

In [23]:
full_model.reset_states()

In [23]:
del full_model

In [21]:
opt = tk.optimizers.Adam(learning_rate=0.01)

In [26]:
full_model.compile(loss='binary_crossentropy',
        optimizer='adam',
        metrics=['acc'])
history2 = full_model.fit([data_train_text,data_train_title],labels_train,epochs = 20,batch_size = 256, 
                     validation_data = [[data_val_text,data_val_title],labels_val],shuffle=True)

Train on 7380 samples, validate on 821 samples
Epoch 1/20
7380/7380 [==============================] - 43s 6ms/sample - loss: 0.7142 - acc: 0.5205 - val_loss: 0.6982 - val_acc: 0.5713
Epoch 2/20
7380/7380 [==============================] - 40s 5ms/sample - loss: 0.6930 - acc: 0.5463 - val_loss: 0.6842 - val_acc: 0.5822
Epoch 3/20
7380/7380 [==============================] - 40s 5ms/sample - loss: 0.6844 - acc: 0.6044 - val_loss: 0.6779 - val_acc: 0.5840
Epoch 4/20
7380/7380 [==============================] - 41s 6ms/sample - loss: 0.6796 - acc: 0.5654 - val_loss: 0.6724 - val_acc: 0.5950
Epoch 5/20
7380/7380 [==============================] - 41s 6ms/sample - loss: 0.6753 - acc: 0.5909 - val_loss: 0.6640 - val_acc: 0.6912
Epoch 6/20
7380/7380 [==============================] - 39s 5ms/sample - loss: 0.6716 - acc: 0.6153 - val_loss: 0.6607 - val_acc: 0.7406
Epoch 7/20
7380/7380 [==============================] - 39s 5ms/sample - loss: 0.6698 - acc: 0.5902 - val_loss: 0.6599 - val_acc: 0

In [28]:
history2 = full_model.fit([data_train_text,data_train_title],labels_train,epochs = 10,batch_size = 256, 
                     validation_data = [[data_val_text,data_val_title],labels_val],shuffle=True)

Train on 7380 samples, validate on 821 samples
Epoch 1/10
7380/7380 [==============================] - 39s 5ms/sample - loss: 0.6040 - acc: 0.7609 - val_loss: 0.5959 - val_acc: 0.7844
Epoch 2/10
7380/7380 [==============================] - 39s 5ms/sample - loss: 0.6109 - acc: 0.7418 - val_loss: 0.5943 - val_acc: 0.7101
Epoch 3/10
7380/7380 [==============================] - 39s 5ms/sample - loss: 0.5841 - acc: 0.7175 - val_loss: 0.5490 - val_acc: 0.7558
Epoch 4/10
7380/7380 [==============================] - 39s 5ms/sample - loss: 0.5651 - acc: 0.7179 - val_loss: 0.5449 - val_acc: 0.7485
Epoch 5/10
7380/7380 [==============================] - 42s 6ms/sample - loss: 0.5651 - acc: 0.7116 - val_loss: 0.5377 - val_acc: 0.7582
Epoch 6/10
7380/7380 [==============================] - 39s 5ms/sample - loss: 0.5684 - acc: 0.6976 - val_loss: 0.5396 - val_acc: 0.7734
Epoch 7/10
7380/7380 [==============================] - 41s 5ms/sample - loss: 0.5664 - acc: 0.7303 - val_loss: 0.6737 - val_acc: 0

In [21]:
conv_model.save("./Checkpoints/conv_model2.h5")

In [22]:
conv_model.save_weights("./Checkpoints/conv_weights2")

In [29]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

pred = full_model.predict([x_test_text,x_test_title])
pred = [np.argmax(x) for x in pred]

print(f"F1-score : {f1_score([np.argmax(x) for x in y_test],pred,pos_label=1)}")
print(f"Accuracy-score : {accuracy_score([np.argmax(x) for x in y_test],pred)}")

print(f"Precision : {precision_score([np.argmax(x) for x in y_test],pred,pos_label = 1)}")
print(f"Recall  : {recall_score([np.argmax(x) for x in y_test],pred,pos_label = 1)}")

F1-score : 0.6365979381443297
Accuracy-score : 0.7250121891760117
Precision : 0.96484375
Recall  : 0.475


In [13]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

pred = full_model.predict([x_test_text,x_test_title])
pred = [np.argmax(x) for x in pred]

print(f"F1-score : {f1_score([np.argmax(x) for x in y_test],pred,pos_label=1)}")
print(f"Accuracy-score : {accuracy_score([np.argmax(x) for x in y_test],pred)}")

print(f"Precision : {precision_score([np.argmax(x) for x in y_test],pred,pos_label = 1)}")
print(f"Recall  : {recall_score([np.argmax(x) for x in y_test],pred,pos_label = 1)}")

F1-score : 0.8322082931533269
Accuracy-score : 0.8303266699171136
Precision : 0.8346228239845261
Recall  : 0.8298076923076924


In [29]:
from sklearn.metrics import accuracy_score

pred = conv_model.predict(x_test)
pred = [np.argmax(x) for x in pred]



In [30]:
from sklearn.metrics import f1_score
f1_score([np.argmax(x) for x in y_test],pred,pos_label=1)

0.9362930077691454

In [31]:
accuracy_score([np.argmax(x) for x in y_test],pred)

0.9093779602147143

In [32]:
from sklearn.metrics import roc_auc_score
roc_auc_score([np.argmax(x) for x in y_test],pred)

0.8797661587489404

In [34]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


print(f"Precision : {precision_score([np.argmax(x) for x in y_test],pred,pos_label = 0)}")
print(f"Recall  : {recall_score([np.argmax(x) for x in y_test],pred,pos_label = 0)}")

Precision : 0.8851894374282434
Recall  : 0.8048016701461378


In [2]:
import pickle
def save_pickle_file(file,file_path,protocol=None):
    with open(file_path,"wb") as f:
        if protocol:
            pickle.dump(file,f,protocol = protocol)
        else:
            pickle.dump(file,f)

def load_pickle_file(file_path, protocol=None):
     with open(file_path,"rb") as f:
        if protocol:
            return pickle.load(f)
        else:
            return pickle.load(f)
